In [11]:
'''
tcloud中用销售订单生成税局发票模板,选择单个销售订单，单独生成相应的发票模板

'''
import os
import re
import easygui
import openpyxl
import numpy as np
import pandas as pd
import shutil

In [12]:
#读取发票模板
fname_fapiao = r"F:\repos\fish\发票模板.xlsx"
df_fapiao = pd.read_excel(fname_fapiao,header = 2,dtype = {'商品和服务税收分类编码':'str'})
df_fapiao

,项目名称,商品和服务税收分类编码,规格型号,单位,商品数量,商品单价,金额,税率,折扣金额,优惠政策类型


In [13]:

path = r"F:\repos\fisha\莱新销售订单0826-0925"
os.chdir(path)

#以选择的销售订单所在文件夹作为工作目录，并在此目录下建“发票模板”文件夹
path1 = '发票模板'
if not os.path.exists(path1):
    try:
        os.mkdir(path1)
    except:
        pass

    


In [14]:
def getPivot(df):
    bianba_fenlei = dict(zip(df['存货编码'],df['存货分类']))
    bianba_mingchen = dict(zip(df['存货编码'],df['存货名称']))
    bianba_daima = dict(zip(df['存货编码'],df['存货代码']))
    bianba_hanlian = dict(zip(df['存货编码'],df['件含量']))
    pivot = df.pivot_table(index = '存货编码',values = ['数量','数量（件）','含税金额'] ,aggfunc = 'sum')
    pivot = pivot.reset_index()
    pivot = pivot.assign(fenlei = pivot['存货编码'].map(bianba_fenlei))
    pivot = pivot.assign(mingchen = pivot['存货编码'].map(bianba_mingchen))
    pivot = pivot.assign(daima = pivot['存货编码'].map(bianba_daima))
    pivot = pivot.assign(hanliang = pivot['存货编码'].map(bianba_hanlian))
    dic = dict(zip(['fenlei','mingchen','daima','hanliang'],['存货分类','存货名称','存货代码','件含量']))
    pivot = pivot.rename(columns =  dic)
             
    
    return pivot

def chuliMingchen(d):
    d['存货名称'] = d['存货名称'].str.replace('运费',"")
    d = d.assign(qian = d['存货名称'].str.split('型').str[0] + '型')
    d = d.assign(qian = d['存货名称'].str.split('页').str[0] + '页')
    d = d.assign(hou = d['存货名称'].str.split('型').str[1])
    d = d.assign(hou = d['存货名称'].str.split('页').str[1])
    d['qian'] = d['qian'].fillna('')
    d['hou'] = d['hou'].fillna('')
    # qian = pd.Series(qian).fillna('')
    # hou = pd.Series(hou).fillna('')
    return d
def getFapiaoBen(d):
    d['项目名称'] = d['存货编码'] + '-' + d['hou']
    d['项目名称'] = d['项目名称'].str.replace('运费-','运费')
    # d['项目名称'] = d['项目名称'].str.split('-').str[0]
    d['商品和服务税收分类编码'] = '1060202010000000000'
    d['规格型号'] = d['qian']
    d['单位'] = '本'
    d['商品数量'] = d['数量']
    d['商品单价'] = ''
    d['金额'] = d['含税金额']
    d['税率'] = 0.13
    d['折扣金额'] = ''
    d['优惠政策类型'] = ''
    d = d[['项目名称',
     '商品和服务税收分类编码',
     '规格型号',
     '单位',
     '商品数量',
     '商品单价',
     '金额',
     '税率',
     '折扣金额',
     '优惠政策类型',
                           ]]
    return d

# gongsi = '热脉'       #获取简称
def getFapiaoMoban(gongsi):
    filename=''.join(['发票模板-',gongsi,'.xlsx'])
    newname = os.path.join(path1,filename)
    shutil.copyfile(fname_fapiao, newname)
    return newname

def fengefu(string):
    num = len(string.split('-'))
    if num  == 3 :
        string = '-'.join([string.split('-')[0],string.split('-')[1]])
        
    elif num == 2 :
        if len(string.split('-')[0]) <= 4:
            string = string
        else :
            string = string.split('-')[0]
    else :
        string = string
    return string
            
    
    

In [15]:
#莱新销售订单超5000条，不能一次导出，分三次导出，并分别存于同一文件下，先将它们合并
data = []
for i in os.listdir(path):
    if  os.path.isfile(i):
        i = os.path.join(path,i)
        df = pd.read_excel(i)
        data.append(df)
    else :
        continue
   
  
    
df_xiaoshou0 = pd.concat(data)
df_xiaoshou1 = df_xiaoshou0.loc[df_xiaoshou0['单据执行状态'] != '合计']
lst1 = ['单据编号',
  '单据日期',
 '含税总金额',
 '存货名称',
 '存货分类',
 '存货编码',
  '存货代码',
  '数量',
  '件含量',
 '数量（件）',
  '含税单价',
 '含税金额',
        '客户'
]
df_xiaoshou2 = df_xiaoshou1[lst1]
df_xiaoshou3 = df_xiaoshou2.copy()
s = []
for i in df_xiaoshou2['存货编码'].to_list():
    j = fengefu(i)
    s.append(j)
df_xiaoshou3['存货编码'] = s

In [16]:
#先调试一个 '武汉市热脉文体用品有限公司'
# df0 = df_xiaoshou2[df_xiaoshou2['客户'] == '武汉市热脉文体用品有限公司'] 

gp = df_xiaoshou3.groupby('客户')
for k,v  in gp:
    gongsi = k[:8]
    newname = getFapiaoMoban(gongsi)
    pivot = getPivot(v)
    pivot1 = chuliMingchen(pivot)
    fapiao = getFapiaoBen(pivot1)
    with pd.ExcelWriter(newname, engine='openpyxl',mode='a', if_sheet_exists='overlay')  as writer:
        fapiao.to_excel(writer, sheet_name = '1-明细模板',startrow=3, header = False,index = False)
    
    
    
    
    
    
    
    
    
    


C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\work

In [ ]:
import pandas as pd
 
# 示例数据
df = pd.DataFrame({
    'A': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'],
    'B': ['one', 'one', 'two', 'two', 'one', 'one'],
    'C': ['small', 'large', 'large', 'small', 'small', 'large'],
     'F': ['small1', 'large1', 'large1', 'small1', 'small1', 'large1'],
    'D': [1, 2, 2, 3, 3, 4],
    'E': [2, 4, 5, 5, 6, 6]
})
 
df

In [ ]:
import pandas as pd
 
# 示例数据
df = pd.DataFrame({
    'A': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'],
    'B': ['one', 'one', 'two', 'two', 'one', 'one'],
    'C': ['small', 'large', 'large', 'small', 'small', 'large'],
    'D': [1, 2, 2, 3, 3, 4],
    'E': [2, 4, 5, 5, 6, 6]
})
 
# 创建透视表，并使用margins参数汇总
pivot_table = df.pivot_table(values='D', index=['A', 'B'], columns=['C'], aggfunc='sum', margins=True)
 
# 将多层索引转换为常规列，并将不是汇总的字段展开
melted = pivot_table.reset_index().melt(id_vars=['A', 'B'], value_vars=pivot_table.columns.levels[1].unique(), var_name='C')
 
# 将汇总行转换为常规列
melted['C'] = melted['C'].str.split('(').str[0]
 
# 重置索引
melted = melted.dropna(subset=['C']).reset_index(drop=True)
 
print(melted)

In [ ]:
import pandas as pd

# 创建示例数据集
data = {
    'A': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'],
    'B': ['one', 'one', 'two', 'two', 'one', 'one'],
    'C': [1, 2, 3, 4, 5, 6],
    'D': [7, 8, 9, 10, 11, 12]
}

df = pd.DataFrame(data)

# 创建透视表
pivot_table = pd.pivot_table(df, values='C', index='A', columns='B', aggfunc='sum', fill_value=0)
print(pivot_table)


In [ ]:
df = pd.read_clipboard()
df

In [ ]:
df['fengeshu'] = df['存货编码'].str.split('-').str.len()

In [ ]:
df['fengeshu'] 

In [ ]:
df.to_excel('fenge.xlsx',index = False)

In [ ]:

    
    
    
    # df['fengeshu'] = df['存货编码'].str.split('-').str.len()
    # df['bianma1'] = df['存货编码']
    # df = df.assign(bianma1  = np.where(df['fenggeshu'] == 1, df['存货编码'],np.where(df['fenggeshu'] == 2,)))
    
    

In [ ]:
s = []
for i in df['存货编码'].to_list():
    print(i)
    
    j = fengefu(i)
    print('j',j)
    s.append(j)
s


In [ ]:
s

In [ ]:
len(s)

In [ ]:
len(df['存货编码'])

In [ ]:
df

In [ ]:
df['存货编码1'] = s

In [ ]:
df.to_excel('fengehou.xlsx',index = False)

In [ ]:
df_xiaoshou3

In [7]:
pivot

,存货编码,含税金额,数量,数量（件）,存货分类,存货名称,存货代码,件含量
0,8K160g素描纸,391.02,120,1.0,锐意美术系列,8K160克20张素描纸,锐意,120.0
1,EXWGB5100Pa,324.00,120,1.0,PP线环本,B5-100型PP线环网格本（侧翻）,电商,120.0
2,FL-3260A,268.66,280,1.0,锐意缝线本,32K60型精品缝线本线条A2302,锐意,280.0
3,FZY-3260B,268.66,280,1.0,锐意缝线本,32K60型精品缝线作业记录本B2302,锐意,280.0
4,N0670,349.20,480,1.0,无线胶装软抄,A530型卡面无线胶订本,抄本,480.0
5,N0671,270.05,320,1.0,无线胶装软抄,A540型卡面无线胶订本,抄本,320.0
6,N0672,284.02,240,1.0,无线胶装软抄,A560型卡面无线胶订本,抄本,240.0
7,N0673,281.11,180,1.0,无线胶装软抄,A580型卡面无线胶订本,抄本,180.0
8,N0783,1650.00,2000,10.0,莱特牛皮缝线本,16K60型牛卡缝线英语本,抄本,200.0
9,N0784（400格）,825.00,1000,5.0,莱特牛皮缝线本,16K60型牛卡缝线作文本,抄本,200.0


In [66]:
df1 = pd.read_clipboard()
df1


,项目名称,商品和服务税收分类编码,规格型号
0,A4图画本（混版）a-,1060202010000000000,A4图画本（混版）a页
1,DJL100混装-,1060202010000000000,100页精品道林胶套本页
2,DJL80混装-,1060202010000000000,80K页精品道林胶套本页
3,EJL16150-,1060202010000000000,16K150型卡面胶套本页
4,EJL1680-,1060202010000000000,16K80型卡面胶套本页
5,EJRJ3280-,1060202010000000000,32K80型卡面学生日记页
6,EJY3280-,1060202010000000000,32K80型卡面胶套英语本页
7,EXBA4100P-,1060202010000000000,A4-100型PP线环空白格本（侧翻）页
8,EXBA6100P-,1060202010000000000,A6-100型PP线环空白本（侧翻）页
9,EXLA4100P-,1060202010000000000,A4-100型PP线环线条本（侧翻）页


In [70]:
# guiges_qian = []
# guiges_hou = []
def guige(string):
    if string.count('型') == 0:
        qian = ''
        hou = ''
    elif string.count('页') == 0:   
        qian = ''
        hou = ''
    else :
                
        
        
        
    if "型" in string :
        if string.endswith('型') != True:
            if string.startswith('型') != True :
                qian = string.split('型')[0] + '型'
                hou =  string.split('型')[1] 
            elif string.startswith('型') == True :
                qian = string.split('型')[1] + '型'
                hou =  string.split('型')[2] 
            else :
                qian = ''
                hou = ''
        else:
            qian = ''
            hou = ''
            

   
    elif  "页" in string :
        if string.endswith('页') != True:
            if string.startswith('页') != True :
                qian = string.split('页')[0] + '页'
                hou =  string.split('页')[1] 
            elif string.startswith('页') == True :
                qian = string.split('页')[1] + '页'
                hou =  string.split('页')[2] 
            else :
                qian = ''
                hou = ''
        else :
             qian = ''
             hou = ''
             
    
              
    else :
        qian = ''
        hou = ''
    
    
    return qian,hou
    
        
    
        
    

In [43]:
qian,hou = guige('aa型aa')
qian
hou

'aa'

In [37]:
"型" in 'aa型aa型'

True

In [40]:
'aa型aa'.startswith('型') != True

True

In [80]:
'型aa型aa型'.count('型')

3

In [71]:
guiges_qian = []
guiges_hou = []
for i in df1['规格型号'].to_list():
    qian,hou = guige(i)
    print(qian,hou)
    guiges_qian.append(qian)
    guiges_hou.append(hou)


 
 
 
16K150型 卡面胶套本页
16K80型 卡面胶套本页
32K80型 卡面学生日记页
32K80型 卡面胶套英语本页
A4-100型 PP线环空白格本（侧翻）页
A6-100型 PP线环空白本（侧翻）页
A4-100型 PP线环线条本（侧翻）页
A6-100型 PP线环横线本（侧翻）页
A6-100型 PP线环网格本（侧翻）页
32K80型 办公无线胶订本页


In [75]:
guiges_qian

['',
 '',
 '',
 '16K150型',
 '16K80型',
 '32K80型',
 '32K80型',
 'A4-100型',
 'A6-100型',
 'A4-100型',
 'A6-100型',
 'A6-100型',
 '32K80型']

In [76]:
guiges_hou

['',
 '',
 '',
 '卡面胶套本页',
 '卡面胶套本页',
 '卡面学生日记页',
 '卡面胶套英语本页',
 'PP线环空白格本（侧翻）页',
 'PP线环空白本（侧翻）页',
 'PP线环线条本（侧翻）页',
 'PP线环横线本（侧翻）页',
 'PP线环网格本（侧翻）页',
 '办公无线胶订本页']

In [73]:
len(guiges_hou)

13